In [33]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
utility_check["account_name"] = "utility"
umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
umbrella_check["account_name"] = "umbrella"
savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
savings_check["account_name"] = "savings"
primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/PrimaryCheck.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
primary_check["account_name"] = "primary"
platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
platinum_card["account_name"] = "platinum"
cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard.csv", header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
cashwise_card["account_name"] = "cashwise"

accounts_joined = utility_check.append(umbrella_check)
accounts_joined = accounts_joined.append(savings_check)
accounts_joined = accounts_joined.append(primary_check)
accounts_joined = accounts_joined.append(platinum_card)
accounts_joined = accounts_joined.append(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check) + len(platinum_card) + len(cashwise_card)



In [34]:
display(accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount")[["Amount", "spending","income"]])
display(accounts_joined["Amount"].sum())


,Amount,spending,income
Date,,,
2021-01-31,-3262.54,-16030.75,12768.21
2021-02-28,-9186.00,-22079.10,12893.10
2021-03-31,-6911.47,-39904.59,32993.12
2021-04-30,66655.41,-35636.79,102292.20
2021-05-31,304.59,-13254.76,13559.35
2021-06-30,-1369.97,-15697.55,14327.58
2021-07-31,-6443.26,-67095.63,60652.37
2021-08-31,-5549.34,-19310.10,13760.76
2021-09-30,-8070.01,-21731.82,13661.81


24284.68

In [16]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))



,Unnamed: 8,Amount_value
Date,,
2021-01-31,0.0,10837.13
2021-02-28,0.0,17298.12
2021-03-31,0.0,24675.13
2021-04-30,0.0,30833.68
2021-05-31,0.0,8434.56
2021-06-30,0.0,11035.41
2021-07-31,0.0,62131.44
2021-08-31,0.0,14658.82
2021-09-30,0.0,17228.95


,Amount,CheckNum
Date,,
2021-01-31,-10837.13,0.0
2021-02-28,-17298.12,253.0
2021-03-31,-24675.13,387.0
2021-04-30,-30833.68,396.0
2021-05-31,-8434.56,0.0
2021-06-30,-11035.41,134.0
2021-07-31,-62131.44,136.0
2021-08-31,-14658.82,0.0
2021-09-30,-17228.95,198.0


## Show month

In [4]:

month = 4

start_date = '2021-{}-01'.format(month)
if month < 12:
    end_date = '2021-{}-01'.format(month+1)
else:
    end_date = '2022-01-01'
end_of_year[(end_of_year["Date"] >= start_date) & (end_of_year["Date"] < end_date)].sort_values("Date")




,Master Category,Subcategory,Date,Location,Payee,Description,Payment Method,Amount,Unnamed: 8,Amount_value
289,Outgoing Transfers,Other Outgoing Transfers,2021-04-01,,NaN,RECURRING TRANSFER TO CLARK J REF #OP0B63HJPH CHECKING MORTGAGE PAYMENT,Primary ck acct ...9865,"$2,091.47",NaN,2091.47
174,Uncategorized Payments,Electronic Payments,2021-04-01,,DOVENMUEHLE MTG MORTG,DOVENMUEHLE MTG MORTG PYMT 040121 1465323044 John P Clark III,Utility ck acct ...9873,"$2,091.47",NaN,2091.47
38,Business/Office,Other Business/Office,2021-04-02,,VENMO,PURCHASE AUTHORIZED ON 04/01 VENMO 855-812-4430,Debit Card/ATM Card ...4775,$32.95,NaN,32.95
39,Business/Office,Other Business/Office,2021-04-02,,VENMO,PURCHASE AUTHORIZED ON 04/01 VENMO 855-812-4430,Debit Card/ATM Card ...4775,$25.00,NaN,25.00
173,Uncategorized Payments,Electronic Payments,2021-04-02,,RURAL INDIA MINI CONTRIBUTN,"RURAL INDIA MINI CONTRIBUTN 210402 XX014IAJ91L5NW CLARK, JOHN",Utility ck acct ...9873,$800.00,NaN,800.00
288,Outgoing Transfers,Other Outgoing Transfers,2021-04-02,,NaN,ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873 REF #IB0B6HLCZ5 ON 04/02/21,Primary ck acct ...9865,"$1,000.00",NaN,1000.00
287,Outgoing Transfers,Other Outgoing Transfers,2021-04-05,,NaN,RECURRING TRANSFER TO CLARK J REF #OP0B6VKZ85 CHECKING TRANFER FOR RIM TITHE,Primary ck acct ...9865,$800.00,NaN,800.00
216,Credit Card/Loan Payments,Wells Fargo Credit Card Payments,2021-04-05,,WELLS FARGO CREDIT CARD,ONLINE TRANSFER REF #IB0B6YLHLF TO PLATINUM CARD XXXXXXXXXXXX0729 ON 04/04/21,Primary ck acct ...9865,$151.21,NaN,151.21
151,Uncategorized Payments,Checks Written,2021-04-06,,NaN,CHECK,Primary ck acct ...9865,"$8,170.00",NaN,8170.00
107,Miscellaneous,Charity/Community Org,2021-04-07,"Redwood City ,CA",GOFNDME HELP FOR KIDS BA,"GoFndMe* help for kids ba Redwood City ,CA",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$6.75,NaN,6.75


,Master Category,Subcategory,Date,Location,Payee,Description,Payment Method,Amount,Unnamed: 8,Amount_value
0,Auto/Transportation,Gasoline,2021-03-14,"FLAGSTAFF ,AZ",FRYS,"FRYS #7104 FLAGSTAFF ,AZ",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$27.68,NaN,27.68
1,Bills/Utilities,Cable/Satellite TV,2021-07-23,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
2,Bills/Utilities,Cable/Satellite TV,2021-06-23,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
3,Bills/Utilities,Cable/Satellite TV,2021-05-24,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
4,Bills/Utilities,Cable/Satellite TV,2021-04-23,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
5,Bills/Utilities,Cable/Satellite TV,2021-03-24,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
6,Bills/Utilities,Cable/Satellite TV,2021-02-23,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
7,Bills/Utilities,Cable/Satellite TV,2021-01-23,"877-794-2724 ,NY",SUDDENLINK,"SUDDENLINK 7715 877-794-2724 ,NY",WELLS FARGO CASH WISE VISA SIGNATURE(R) CARD ...0560,$69.41,NaN,69.41
8,Bills/Utilities,Gas/Electric,2021-12-20,,UNS GAS,UNS Gas Corporat SNAP PMT 211216 8327050537 HANNAH GENIECE CLARK,Umbrella ...7135,$145.21,NaN,145.21
9,Bills/Utilities,Gas/Electric,2021-12-15,,APS ELECTRIC,APS electric pmt PAYMENT 211213 3678521956 Autopay,Umbrella ...7135,$106.69,NaN,106.69
